In [240]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [241]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data

--2025-10-13 23:49:32--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.11’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.001s  

2025-10-13 23:49:32 (65.5 MB/s) - ‘course_lead_scoring.csv.11’ saved [80876/80876]



In [242]:
df = pd.read_csv(data)
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [243]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [244]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)
categorical


['lead_source', 'industry', 'employment_status', 'location']

In [245]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [246]:
for c in categorical:
    df[c] = df[c].fillna('NA')

In [247]:
df['annual_income'] = df['annual_income'].fillna(0.0)

In [248]:
df['industry'].mode()[0]

'retail'

In [249]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count']
df[numerical].corrwith(df.lead_score).abs()

number_of_courses_viewed    0.004879
annual_income               0.015610
interaction_count           0.009888
dtype: float64

In [250]:
df[numerical].corrwith(df.interaction_count).abs()

number_of_courses_viewed    0.023565
annual_income               0.027036
interaction_count           1.000000
dtype: float64

In [251]:
from sklearn.model_selection import train_test_split

In [252]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [253]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [254]:
y_train = df_train.converted
y_val = df_val.converted
y_test = df_test.converted

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [255]:
from sklearn.metrics import mutual_info_score

In [256]:
round(mutual_info_score(df_full_train.industry, df_full_train.converted),2)

0.01

In [257]:
round(mutual_info_score(df_full_train.location, df_full_train.converted),2)

0.0

In [258]:
round(mutual_info_score(df_full_train.lead_source, df_full_train.converted),2)

0.03

In [259]:
round(mutual_info_score(df_full_train.employment_status, df_full_train.converted),2)

0.01

In [260]:
#df.lead_score.nunique()

### One Hot Encoding

In [261]:
from sklearn.feature_extraction import DictVectorizer

In [262]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

## Train a logistic regression

In [265]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [266]:
# Predict on validation set
y_pred = model.predict(X_val)
# Calculate accuracy
acc = accuracy_score(y_val, y_pred)

# Round to 2 decimals
print(round(acc, 2))

0.71


In [270]:
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = {}

for c in C_values:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    accuracies[c] = round(acc, 3)

print(accuracies)

{0.01: 0.696, 0.1: 0.7, 1: 0.706, 10: 0.706, 100: 0.706}
